In [ ]:
import logging
import json
import torch

from vit_pytorch import ViT
from einops import rearrange, repeat
from einops.layers.torch import Rearrange
from collections import OrderedDict
from typing import Tuple

x = torch.randn(size=(64,3,180,320))
x.shape

In [ ]:
!ll